# Poisson equation

c.f. https://teamcoil.sp.u-tokai.ac.jp/lectures/EL1/Poisson/index.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import functools
import pickle

In [ ]:
N = 100
X = 1.0
e0 = 8.85e-12
center = np.array([N // 2, N // 2])
delta = X / N
Conv = 1.0e-6
phi = np.zeros([N, N])
rho = np.zeros([N, N])

In [ ]:
for i in range(N):
    for j in range(N):
        if np.linalg.norm(center - (i, j))*delta < 0.05:
            rho[i, j] = 1.0e-8

In [ ]:
# Eq. (6)
def calc_phi_at(i, j, phi: np.ndarray, rho: np.ndarray, e0):
    return 0.25*(rho[i, j]*(delta**2)/e0+phi[i+1, j]+phi[i-1, j]+phi[i, j+1]+phi[i, j-1])

calc_phi_at_ = functools.partial(calc_phi_at, rho=rho, e0=e0)

In [ ]:
%%time

loop = 0
MaxPhi = 1.0e-10
while True:
    if loop%1000 == 0:
        print(f'{loop=} {MaxPhi=}')

    MaxErr = CurErr = 0
    for i in range(1, N-1):
        for j in range(1, N-1):
            Prev_phi = phi[i, j]
            phi[i, j] = calc_phi_at_(i, j, phi)

            if MaxPhi < abs(phi[i, j]):
                MaxPhi = phi[i, j]

            CurErr = abs(phi[i, j] - Prev_phi) / MaxPhi

            if MaxErr < CurErr:
                MaxErr = CurErr
    loop += 1
    if MaxErr <= Conv:
        break

In [ ]:
with open('rho.pkl', 'wb') as fout:
    pickle.dump(rho, fout)
with open('phi.pkl', 'wb') as fout:
    pickle.dump(phi, fout)

In [ ]:
fig, ax = plt.subplots()
xs, ys = np.meshgrid(np.arange(N), np.arange(N))
zs = phi[xs, ys]
xs, ys = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
im = ax.pcolormesh(xs, ys, zs, vmin=np.min(phi), vmax=np.max(phi), cmap='rainbow') # or jet
fig.colorbar(im, ax=ax)
ax.set_aspect('equal')
plt.show()